In [10]:
import cv2
import numpy as np
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras.models import Model
from keras.layers import Dense
from keras.preprocessing import image

# Load pre-trained VGGFace model
vggface_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Add additional dense layers for age and gender prediction
output = Dense(101, activation='softmax', name='age_output')(vggface_model.layers[-1].output)
output = Dense(2, activation='softmax', name='gender_output')(output)

# Create a model
model = Model(inputs=vggface_model.input, outputs=[output])

# Load pre-trained weights
model.load_weights('vggface_age_gender_model.h5')

# Load haarcascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect age and gender from image
def detect_age_gender(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        face_img = img[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (224, 224))
        face_img = image.img_to_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        face_img = preprocess_input(face_img)
        
        # Predict age and gender
        predictions = model.predict(face_img)
        age_pred = np.argmax(predictions[0])
        gender_pred = np.argmax(predictions[1])
        
        # Age
        predicted_age = age_pred + 1  # Ages are from 0-100
        print("Predicted Age:", predicted_age)
        
        # Gender
        gender = "Male" if gender_pred == 1 else "Female"
        print("Predicted Gender:", gender)
        
        # Draw rectangle around detected face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Display age and gender text
        text = f"Age: {predicted_age}, Gender: {gender}"
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function with an image
detect_age_gender('test_image.jpg')


ImportError: cannot import name 'layer_utils' from 'keras.utils' (c:\Users\Tejashree\OneDrive\Desktop\DL\myvenv\Lib\site-packages\keras\api\utils\__init__.py)